In [27]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
import uuid
from ollama import Client

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [3]:
openai = OpenAI()

In [4]:
'''
Serão testados quatro tipos de prompt:

    --> Zero-shot prompting
    --> Few-shot prompting
    --> Chain-of-Thought prompting
    --> Role-Playing prompting

'''

'\nSerão testados quatro tipos de prompt:\n\n    --> Zero-shot prompting\n    --> Few-shot prompting\n    --> Chain-of-Thought prompting\n    --> Role-Playing prompting\n\n'

In [5]:
'''

---------------------------------------- Zero-shot prompting --------------------------------------

'''

'\n\n---------------------------------------- Zero-shot prompting --------------------------------------\n\n'

In [8]:
prompt = """
Provide a non-functional requirements recommendation for this project:

[
    {
        "fr": "Como administrador, desejo editar permissões de acesso a relatórios financeiros.",
        "dominio": "Office/Business",
        "plataforma": "Web",
        "arquitetura": "Client-server",
        "modulo": "Autenticação",
        "operacao": "Editar_permissão",
        "tecnologias": {"Angular", "TypeScript", "HTML", "CSS"}
    }
]
"""

In [9]:
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": prompt}
        ]
    )
print("\n--- RESPOSTA ---\n")
print(response.choices[0].message.content)


--- RESPOSTA ---

For the project you described, here are some recommended non-functional requirements:

1. **Performance**:
   - The system should respond to permission edit requests within 2 seconds under normal load conditions. This ensures that administrators can efficiently manage access without experiencing delays.

2. **Scalability**:
   - The application should be able to support up to 100 concurrent administrators editing permissions simultaneously without degradation in performance.

3. **Security**:
   - All data transmissions related to editing permissions must be encrypted using TLS 1.2 or higher to protect sensitive financial data from unauthorized access.
   - Role-based access control (RBAC) must be enforced to ensure that only authorized administrators can edit permissions.

4. **Usability**:
   - The interface for editing permissions should adhere to established UX design principles, ensuring that 90% of users can complete tasks without external help during usability

In [10]:
'''

---------------------------------------- Few-shot prompting --------------------------------------

'''

'\n\n---------------------------------------- Few-shot prompting --------------------------------------\n\n'

In [13]:
prompt = """
Provide a non-functional requirements recommendation for this project:

[
    {
        "fr": "Como administrador, desejo editar permissões de acesso a relatórios financeiros.",
        "dominio": "Office/Business",
        "plataforma": "Web",
        "arquitetura": "Client-server",
        "modulo": "Autenticação",
        "operacao": "Editar_permissão",
        "tecnologias": {"Angular", "TypeScript", "HTML", "CSS"}
    }
]

The recommendation must be in this format:
[
  {
    "Tarefa_ID": "TAxx",
    "NFR_Tipo": "Reliability",
    "NFR_Atributo": "availability",
    "NFR_Sentença": "The system must notify users when the function is unavailable."
  }
]
"""

In [14]:
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a requirements engineer"},
        {"role": "user", "content": prompt}
        ]
    )
print("\n--- RESPOSTA ---\n")
print(response.choices[0].message.content)


--- RESPOSTA ---

Here is a non-functional requirements recommendation for the specified project:

```json
[
  {
    "Tarefa_ID": "TA01",
    "NFR_Tipo": "Reliability",
    "NFR_Atributo": "availability",
    "NFR_Sentença": "The system must notify users when the Edit permissions function is unavailable."
  },
  {
    "Tarefa_ID": "TA02",
    "NFR_Tipo": "Performance",
    "NFR_Atributo": "response time",
    "NFR_Sentença": "The system must complete the permission editing operation within 2 seconds under normal load."
  },
  {
    "Tarefa_ID": "TA03",
    "NFR_Tipo": "Usability",
    "NFR_Atributo": "user interface",
    "NFR_Sentença": "The editing permissions interface must be intuitive and require no more than three clicks to access the relevant functions."
  },
  {
    "Tarefa_ID": "TA04",
    "NFR_Tipo": "Security",
    "NFR_Atributo": "data protection",
    "NFR_Sentença": "The system must ensure that only authorized users can access and edit financial report permissions."
  },

In [15]:
'''

---------------------------------------- Chain-of-Thought prompting --------------------------------------

'''

'\n\n---------------------------------------- Chain-of-Thought prompting --------------------------------------\n\n'

In [20]:
prompt = """
Given the structured project data below, analyze and recommend one non-functional requirement (NFR) that fits the scenario. Only recommend NFRs belonging to the following types and attributes:

- Performance: response time, capacity, transit delay, efficiency compliance  
- Reliability: availability, integrity, fault tolerance, recoverability  
- Security: confidentiality, access control, authentication  

Follow this step-by-step reasoning:

1. Read the functional and contextual information:
[
    {
        "fr": "Como administrador, desejo editar permissões de acesso a relatórios financeiros.",
        "dominio": "Office/Business",
        "plataforma": "Web",
        "arquitetura": "Client-server",
        "modulo": "Autenticação",
        "operacao": "Editar_permissão",
        "tecnologias": ["Angular", "TypeScript", "HTML", "CSS"]
    }
]

2. Understand the functional requirement (FR):  
   - It involves editing access permissions to financial reports (a sensitive business action).

3. Evaluate the context:
   - Web platform → implies network latency and browser issues.  
   - Sensitive data (financial) → implies strong security needs.  
   - Authentication module → related to access control and integrity.  
   - Client-server → implies fault tolerance and API reliability.

4. Choose the most relevant NFR type and attribute based on this context.

5. Draft one clear NFR sentence, aligned with the selected attribute.  
   - Example: "The system must ensure the [ATTRIBUTE] by [MEASURE OR CONDITION]."

6. Format the final output as JSON in the structure below (only one recommendation):

[
  {
    "Tarefa_ID": "TA01",
    "NFR_Tipo": "[Performance|Reliability|Security]",
    "NFR_Atributo": "[attribute from allowed list]",
    "NFR_Sentença": "..."
  }
]

Now generate the recommendation.
"""


In [28]:
'''
Função para filtrar do csv com os dados de todos os projetos apenas os que contém NFRs atribuídos
'''

def filtrar_linhas_com_nfr_tipo(caminho_csv, caminho_saida):
    try:
        df = pd.read_csv(caminho_csv)

        if "NFR_Tipo" not in df.columns:
            print("A coluna 'NFR_Tipo' não existe no arquivo.")
            return

        df_filtrado = df[df["NFR_Tipo"].notna() & (df["NFR_Tipo"].astype(str).str.strip() != "")]

        if df_filtrado.empty:
            print("Nenhuma linha com 'NFR_Tipo' preenchido foi encontrada.")
        else:
            df_filtrado.to_csv(caminho_saida, index=False)
            print(f"Arquivo salvo em: {caminho_saida}")
    except Exception as e:
        print(f"Erro ao processar o arquivo: {e}")

In [30]:
caminho_entrada = r"C:\Users\Renanzin\llm\llm_nfr\LLM_to_recommend_NFRs\Data\DadosProjetos.csv"
caminho_saida = r"C:\Users\Renanzin\llm\llm_nfr\LLM_to_recommend_NFRs\Data\dadosNFR.csv"

filtrar_linhas_com_nfr_tipo(caminho_entrada, caminho_saida)


Arquivo salvo em: C:\Users\Renanzin\llm\llm_nfr\LLM_to_recommend_NFRs\Data\dadosNFR.csv


In [21]:
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a requirements engineer. Use chain-of-thought reasoning to analyze the user's input."},
        {"role": "user", "content": prompt}
        ]
    )
print("\n--- RESPOSTA ---\n")
print(response.choices[0].message.content)


--- RESPOSTA ---

Based on the analysis of the project data provided, the most relevant non-functional requirement focuses on security due to the sensitivity of financial reports and the need for appropriate access control. Here's the recommendation structured in JSON format:

```json
[
  {
    "Tarefa_ID": "TA01",
    "NFR_Tipo": "Security",
    "NFR_Atributo": "access control",
    "NFR_Sentença": "The system must ensure strict access control by allowing only authorized administrators to edit permissions for financial reports."
  }
]
```
